In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/TinyStories-valid.txt', 'r', encoding='utf-8') as f:
   text=f.read()

print(len(text))


19432979


In [ ]:
import torch

In [ ]:
# -*- coding: utf-8 -*-
"""tinygptforcollab

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/19DiOjHtL-UEa69WlOcMsmRnkrNc5NmaS
"""

# -*- coding: utf-8 -*-
"""tinygpt.ipynb
Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/109yvdricQdl9D43KPCeh829Y89TGUcvC


"""
#hypparameters
device='cuda' if torch.cuda.is_available() else 'cpu'
eval_iters=200
n_embd=384
lr=3e-4
batch_size=64
block_size=256
num_heads=6
eval_inter=500
dropout=0.2
nlayerb=6
max_iters=5000

chars=sorted(list((set(text))))
vocab_size=len(chars)

itos={i:s for i, s in enumerate(chars)}
stoi={s:i for i, s in enumerate(chars)}

encode = lambda e: [stoi[c] for c in e]
decode = lambda d: "".join([itos[c] for c in d])




data=torch.tensor(encode(text))
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]

torch.manual_seed(1337)

def get_batch(split):
    data = train_data if split=='train' else val_data
    ix=torch.randint(len(data)-block_size, (batch_size, ))
    x=torch.stack([data[i:i+block_size]for i in ix])
    y=torch.stack([data[i+1:i+block_size+1]for i in ix])
    return x, y

xb, yb=get_batch('train')

import torch
from torch import nn as nn
import torch.nn.functional as F

class Attention(nn.Module):
   def __init__(self):
    super().__init__()
    self.kqv = nn.Linear(n_embd, 3*n_embd)
    self.ddp=nn.Dropout(dropout)
    self.proj=nn.Linear(n_embd, n_embd)


   def forward(self, x):
      B, T, C = x.shape
      k, q, v = self.kqv(x).split(n_embd, dim=2)
      k = k.view(B, T, num_heads, C//num_heads).transpose(1, 2)
      q = q.view(B, T, num_heads, C//num_heads).transpose(1, 2)
      v = v.view(B, T, num_heads, C//num_heads).transpose(1, 2)
      y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=dropout if self.training else 0, is_causal=True)
      y = y.transpose(1, 2).contiguous().view(B, T, C)
      out=self.ddp(self.proj(y))
      return out



class feedfoward(nn.Module):
  def __init__(self, n_embd):
    super().__init__()
    self.net=nn.Sequential(nn.Linear(n_embd, 4*n_embd), nn.ReLU(), nn.Linear(4*n_embd, n_embd), nn.Dropout(dropout))
  def forward(self, x):
    ffd=self.net(x)
    return ffd

class Block(nn.Module):
  def __init__(self, n_embd, num_heads):
    super().__init__()
    head_size=n_embd//num_heads
    self.heads=Attention()
    self.ffd=feedfoward(n_embd)
    self.lln=nn.LayerNorm(n_embd)
  def forward(self, x):
    x=x+self.heads(self.lln(x))
    x=x+self.ffd(self.lln(x))
    return x







torch.manual_seed(1337)
class BigramModel(nn.Module):
    def __init__ (self):
        super().__init__()
        self.tokenembeddingtable=nn.Embedding(vocab_size, n_embd)
        self.positionembeddingtable=nn.Embedding(block_size, n_embd)
        self.heads=Attention()
        self.ffd=feedfoward(n_embd)
        self.block=nn.Sequential(*[Block(n_embd, num_heads=num_heads)for _ in range(nlayerb)])
        self.lnf=nn.LayerNorm(n_embd)
        self.lmhead=nn.Linear(n_embd, vocab_size)
    def forward(self, idx, targets=None):
        idx = idx.to(device)
        B, T= idx.shape
        tok_emb=self.tokenembeddingtable(idx)
        pos_emb=self.positionembeddingtable(torch.arange(T, device=device))
        x=tok_emb+pos_emb
        x=self.block(x)
        x=self.lnf(x)

        logits=self.lmhead(x)


        B, T, C =logits.shape

        if targets is not None:
          logits=logits.view(B*T, C)
          targets=targets.to(device)

          targets=targets.view(B*T)
          loss=F.cross_entropy(logits, targets)
        else:
          loss=None
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
          idx_cong=idx[:, -block_size:]
          logits, loss=self(idx_cong)
          logits=logits[:, -1, :]
          prob=F.softmax(logits, dim=-1)
          idx_next=torch.multinomial(prob, num_samples=1)
          idx=torch.cat((idx, idx_next), dim=1)


        return idx
@torch.no_grad
def eval():
    bm1.eval()
    out={}
    for split in ['train', 'val']:
      losses=torch.zeros(eval_iters)
      for k in range(eval_iters):
         X, Y=get_batch(split)
         logits, loss= bm1(X, Y)
         losses[k]=loss
      out[split]=losses.mean()
    return out
    bm1.train()












bm1=BigramModel()
bm1=bm1.to(device)

optimizer=torch.optim.AdamW(bm1.parameters(), lr)


for _ in range(max_iters):
  xb, yb=get_batch("train")
  logits, loss=bm1(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
  if _ % eval_inter==0:
     losses=eval()
     print(f"step:{_} train loss:{losses['train']}, val loss:{losses['val']}")
  if _==4999:
    losses=eval()
    print(f"step:{_} train loss:{losses['train']}, val loss:{losses['val']}")




step:0 train loss:3.626471519470215, val loss:3.623002529144287
step:500 train loss:1.458615779876709, val loss:1.4780384302139282
step:1000 train loss:1.1152454614639282, val loss:1.1413525342941284
step:1500 train loss:0.9907596111297607, val loss:1.015985369682312
step:2000 train loss:0.9231754541397095, val loss:0.9497276544570923
step:2500 train loss:0.8725950717926025, val loss:0.9003527760505676
step:3000 train loss:0.8442314267158508, val loss:0.8730030655860901
step:3500 train loss:0.810728907585144, val loss:0.8426480293273926
step:4000 train loss:0.7813295722007751, val loss:0.8122912645339966
step:4500 train loss:0.7651664018630981, val loss:0.7992984652519226
step:4999 train loss:0.7474968433380127, val loss:0.782487154006958


In [ ]:
PATH='/content/drive/MyDrive/flashtinygpttinystories.pt'
torch.save(bm1.state_dict(), PATH)